<a href="https://colab.research.google.com/github/lgiesen/forest-height/blob/main/PreTrained_SemiSupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!unzip gdrive/My\ Drive/DataDa2/images_train.zip

Archive:  gdrive/My Drive/DataDa2/images_train.zip
  inflating: images/image_000.npy    
  inflating: images/image_001.npy    
  inflating: images/image_002.npy    
  inflating: images/image_003.npy    
  inflating: images/image_004.npy    
  inflating: images/image_005.npy    
  inflating: images/image_006.npy    
  inflating: images/image_007.npy    
  inflating: images/image_008.npy    
  inflating: images/image_009.npy    
  inflating: images/image_010.npy    
  inflating: images/image_011.npy    
  inflating: images/image_012.npy    
  inflating: images/image_013.npy    
  inflating: images/image_014.npy    
  inflating: images/image_015.npy    
  inflating: images/image_016.npy    
  inflating: images/image_017.npy    
  inflating: images/image_018.npy    
  inflating: images/image_019.npy    


In [5]:
!unzip gdrive/My\ Drive/DataDa2/masks_train.zip

Archive:  gdrive/My Drive/DataDa2/masks_train.zip
  inflating: masks/mask_000.npy      
  inflating: masks/mask_001.npy      
  inflating: masks/mask_002.npy      
  inflating: masks/mask_003.npy      
  inflating: masks/mask_004.npy      
  inflating: masks/mask_005.npy      
  inflating: masks/mask_006.npy      
  inflating: masks/mask_007.npy      
  inflating: masks/mask_008.npy      
  inflating: masks/mask_009.npy      
  inflating: masks/mask_010.npy      
  inflating: masks/mask_011.npy      
  inflating: masks/mask_012.npy      
  inflating: masks/mask_013.npy      
  inflating: masks/mask_014.npy      
  inflating: masks/mask_015.npy      
  inflating: masks/mask_016.npy      
  inflating: masks/mask_017.npy      
  inflating: masks/mask_018.npy      
  inflating: masks/mask_019.npy      


In [6]:
import numpy as np
import pandas as pd

data = np.empty((0, 9))

for i in range(10):
    X = np.load('/content/images/image_00' + str(i) + '.npy')
    y = np.load('/content/masks/mask_00' + str(i) + '.npy')

    X1 = X[[0, 1, 2, 6, 7, 8, 9]]
    X1 = X1.reshape(7, -1).transpose()

    y = y.reshape(1, -1).transpose()

    # Calculate NDVI
    b8 = X[[6]].reshape(1, -1).transpose()
    b4 = X[[2]].reshape(1, -1).transpose()

    denominator = b8 + b4
    ndvi = np.where(denominator != 0, (b8 - b4) / denominator, 0)


    # Concatenate the slices with the new column

    Xy = np.concatenate((X1, y), axis=1)
    index = Xy.shape[1] - 1

    left_slice = Xy[:, :index]
    right_slice = Xy[:, index:]
    Xy = np.concatenate((left_slice, ndvi, right_slice), axis=1)

    # Concatenate the arrays along the first axis (number of features)
    data = np.concatenate((data, Xy), axis=0)




<ipython-input-6-7eed8fb048c9>:20: RuntimeWarning: invalid value encountered in true_divide
  ndvi = np.where(denominator != 0, (b8 - b4) / denominator, 0)


In [7]:
for i in range(10,20):
    X = np.load('/content/images/image_0' + str(i) + '.npy')
    y = np.load('/content/masks/mask_0' + str(i) + '.npy')
    X1 = X[[0, 1, 2, 6, 7, 8, 9]]
    X1 = X1.reshape(7, -1).transpose()
    y = y.reshape(1, -1).transpose()

    # Calculate NDVI
    b8 = X[[6]].reshape(1, -1).transpose()
    b4 = X[[2]].reshape(1, -1).transpose()
    denominator = b8 + b4
    ndvi = np.where(denominator != 0, (b8 - b4) / denominator, 0)

    # Concatenate the slices with the new column

    Xy = np.concatenate((X1, y), axis=1)
    index = Xy.shape[1] - 1

    left_slice = Xy[:, :index]
    right_slice = Xy[:, index:]
    Xy = np.concatenate((left_slice, ndvi, right_slice), axis=1)
    #Xy_ndvi = np.hstack((Xy, ndvi))

    # Concatenate the arrays along the first axis (number of features)
    data = np.concatenate((data, Xy), axis=0)
    print(data.shape)

(11534336, 9)
(12582912, 9)
(13631488, 9)
(14680064, 9)
(15728640, 9)
(16777216, 9)
(17825792, 9)
(18874368, 9)
(19922944, 9)
(20971520, 9)


In [8]:

num = np.count_nonzero(data[:,-1])
nonzero = np.nonzero(data[:,-1])
zero = np.where(data[:,-1] == 0)[0]


labeled_data = data[nonzero]
unlabeled_data = data[zero]
print(labeled_data.shape)
print(unlabeled_data.shape)
df = pd.DataFrame(labeled_data)
df1 = pd.DataFrame(unlabeled_data)
print(df)
print(df1)



(32239, 9)
(20939281, 9)
           0       1       2       3       4       5       6         7  \
0      217.0   407.0   233.0  2150.0  2036.0  1013.0   515.0  0.804448   
1      233.0   317.0   238.0  1581.0  1684.0   732.0   387.0  0.738318   
2      157.0   287.0   215.0  1381.0  1720.0   839.0   457.0  0.730576   
3      220.0   351.0   268.0  1844.0  2028.0   935.0   502.0  0.746212   
4      254.0   408.0   314.0  2202.0  2689.0  1263.0   654.0  0.750397   
...      ...     ...     ...     ...     ...     ...     ...       ...   
32234  220.0   313.0   236.0  1711.0  1757.0   774.0   362.0  0.757576   
32235  361.0   490.0   365.0  2423.0  2334.0  1096.0   585.0  0.738164   
32236  395.0   538.0   336.0  2790.0  2988.0  1409.0   703.0  0.785029   
32237  346.0   491.0   348.0  2986.0  3333.0  1764.0   928.0  0.791242   
32238  831.0  1060.0  1232.0  2920.0  3208.0  2482.0  1521.0  0.406551   

               8  
0      27.389999  
1      27.200001  
2      25.450001  
3      30.

In [9]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
#X_train, y_train, X_test, y_test = train_test_split(X1,y1, test_size=0.2, random_state=1)

# Step 3: Create training and testing sets
X_train_labeled, X_test_labeled, y_train_labeled, y_test_labeled = train_test_split(
    labeled_data[:, :-1],  # Features
    labeled_data[:, -1],   # Labels
    test_size=0.2,          # Adjust the test size as needed
    random_state=11         # Set a random seed for reproducibility
)




In [10]:

# Split the dataset into training and validation sets
#X_train, y_train, X_test, y_test = train_test_split(X1,y1, test_size=0.2, random_state=1)

# Step 3: Create training and testing sets



X_train_unlabeled= unlabeled_data[:, :-1]

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.semi_supervised import LabelPropagation



# Pre-training step
pretrain_model = RandomForestRegressor()
pretrain_model.fit(X_train_labeled, y_train_labeled)

# Generate representations for labeled and unlabeled data using the pre-trained model
labeled_representations = pretrain_model.predict(X_train_labeled)
unlabeled_representations = pretrain_model.predict(X_train_unlabeled)

# Combine labeled and unlabeled data
combined_data = np.vstack((X_train_labeled, X_train_unlabeled))
combined_representations = np.concatenate((labeled_representations, unlabeled_representations))



In [ ]:
# Create a regression model (e.g., RandomForestRegressor)
regression_model = RandomForestRegressor()
regression_model.fit(combined_data, combined_representations)

# Predict forest height for the test data
y_pred_test = regression_model.predict(X_test_labeled)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

from sklearn.metrics import mean_squared_error

# compute RMSE
print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test_labeled, y_pred_test))))

# visualize predictions vs. true labels
fig = plt.figure(figsize=(8,8))
plt.scatter(y_pred_test, y_test_labeled, color="blue", alpha=0.5)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
plt.plot([-1, 83], [-1, 83], 'k--')
plt.xlabel("Predictions")
plt.ylabel("True Labels")
plt.xlim([-1, 83])
plt.ylim([-1, 83])
plt.title("Evaluation of Random Forest Regression Model ")
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

mse = mean_absolute_error(y_test_labeled, y_pred_test)

print("Mean Absolute Error:", mse)